In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached SQLAlchemy-2.0.23-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached google_cloud_storage-2.14.0-py2.py3-none-any.whl (121 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl (346 kB)
  Using cached greenlet-3.0.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (616 kB)
  Using cached google_api_core-2.15.0-py3-none-any.whl (121 kB)
  Using cached google_crc32c-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)
  Using cached google_auth-2.26.2-py2.py3-none-any.whl (186 kB)
  Using cached google_resumable_media-2.7.0-py2.py3-no

In [1]:
from pathlib import Path
from google.cloud.storage import Client, transfer_manager
from dotenv import load_dotenv
import os

In [2]:
load_dotenv('../aya-data-notebooks/AYA-collection-prep/res/.env.analytics-instruct-multilingual-app.prod')

True

In [3]:
project_id = os.environ.get('C4AI_PROJECT_ID')
bucket_name = os.environ.get('AYA_COLLECTION_BUCKET')
print(bucket_name)

c4ai-instruct-multilingual-data


In [4]:
storage_client = Client(project=project_id)
bucket = storage_client.bucket(bucket_name)

In [5]:
TEMPLATE_BASE_DIR = '/home/shivalikasingh95/aya_collection_plots/templated_datasets/release-v0-vm/'

TEMP_INFO_FILES = 'aya-data-notebooks/AYA-collection-prep/release-data/'

aya_collection_info_dir_templated = f"{TEMP_INFO_FILES}"
aya_collection_data_dir_templated = f"{TEMPLATE_BASE_DIR}data/"

In [6]:
aya_collection_info_dir_translated = "../aya-data-notebooks/AYA-collection-prep/translated-v2/info"

#"../aya-data-notebooks/AYA-collection-prep/translated-collection-v0-backup/info"
aya_collection_data_dir_translated = "../aya-data-notebooks/AYA-collection-prep/translated-v2/data"

#"../aya-data-notebooks/AYA-collection-prep/translated-collection-v0-backup/data"

## Prepare info file path

In [7]:
aya_collection_info_dir = aya_collection_info_dir_translated #aya_collection_info_dir_templated

info_directory_path = Path(aya_collection_info_dir)
print(info_directory_path)

../aya-data-notebooks/AYA-collection-prep/translated-v2/info


In [8]:
# paths = directory_path.rglob("*.jsonl")
# file_paths = [path for path in paths if path.is_file()]
# file_paths
info_file_paths = [x for x in info_directory_path.glob("*.jsonl") if not x.name.startswith(".ipynb_checkpoints")]
info_file_paths

[PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_piqa_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_flan_lambada_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_wikiqa_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_dolly_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_mlqa_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_flan_qa_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_joke_explaination_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_flan_cot_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/info/translated_mintaka_info.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-c

In [9]:
relative_paths = [path.relative_to(info_directory_path) for path in info_file_paths]
relative_paths

[PosixPath('translated_piqa_info.jsonl'),
 PosixPath('translated_flan_lambada_info.jsonl'),
 PosixPath('translated_wikiqa_info.jsonl'),
 PosixPath('translated_dolly_info.jsonl'),
 PosixPath('translated_mlqa_info.jsonl'),
 PosixPath('translated_flan_qa_info.jsonl'),
 PosixPath('translated_joke_explaination_info.jsonl'),
 PosixPath('translated_flan_cot_info.jsonl'),
 PosixPath('translated_mintaka_info.jsonl'),
 PosixPath('translated_cnn_dailymail_info.jsonl'),
 PosixPath('translated_soda_info.jsonl'),
 PosixPath('translated_nqopen_info.jsonl'),
 PosixPath('translated_wiki_split_info.jsonl'),
 PosixPath('translated_flan_coqa_info.jsonl'),
 PosixPath('translated_adversarial_qa_info.jsonl'),
 PosixPath('translated_paws_info.jsonl'),
 PosixPath('translated_xlel_wd_info.jsonl'),
 PosixPath('translated_hotpotqa_info.jsonl'),
 PosixPath('translated_flan_gem_wiki_info.jsonl')]

In [10]:
string_paths = [str(path) for path in relative_paths]

print("Found {} files.".format(len(string_paths)))

Found 19 files.


## Test Pushing Sample files

In [11]:
string_paths

['translated_piqa_info.jsonl',
 'translated_flan_lambada_info.jsonl',
 'translated_wikiqa_info.jsonl',
 'translated_dolly_info.jsonl',
 'translated_mlqa_info.jsonl',
 'translated_flan_qa_info.jsonl',
 'translated_joke_explaination_info.jsonl',
 'translated_flan_cot_info.jsonl',
 'translated_mintaka_info.jsonl',
 'translated_cnn_dailymail_info.jsonl',
 'translated_soda_info.jsonl',
 'translated_nqopen_info.jsonl',
 'translated_wiki_split_info.jsonl',
 'translated_flan_coqa_info.jsonl',
 'translated_adversarial_qa_info.jsonl',
 'translated_paws_info.jsonl',
 'translated_xlel_wd_info.jsonl',
 'translated_hotpotqa_info.jsonl',
 'translated_flan_gem_wiki_info.jsonl']

In [54]:
# test upload with 2 sample files

# v0 path for templated --> "aya-collection-staging/v0/info/"

# v1 path for templated --> "aya-collection-staging/v1/templated_datasets/info/"

# v1 path for translated --> "aya-collection-staging/v1/translated_datasets/info/"

results = transfer_manager.upload_many_from_filenames(
    bucket, string_paths[1:3], source_directory=aya_collection_info_dir, max_workers = 8, blob_name_prefix="aya-collection-staging/v1/translated_datasets/info/")

for name, result in zip(string_paths[1:3], results):
    if isinstance(result, Exception):
            print("Failed to upload {} due to exception: {}".format(name, result))
    else:
        print("Uploaded {} to {}.".format(name, bucket.name))

Uploaded FLAN_CoT_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded joke_explaination_info.jsonl to c4ai-instruct-multilingual-data.


## Push info files

In [12]:
# push all info files

## Check source_directory before upload --> source_directory!!

# v0 path for templated --> "aya-collection-staging/v0/info/"

# v1 path for templated --> "aya-collection-staging/v1/templated_datasets/info/"

# v1 path for translated --> "aya-collection-staging/v1/translated_datasets/info/"

# v2 path for translated --> "aya-collection-staging/v2/translated_datasets/info/"

results = transfer_manager.upload_many_from_filenames(
    bucket, string_paths, source_directory=aya_collection_info_dir, max_workers = 8, blob_name_prefix="aya-collection-staging/v2/translated_datasets/info/")

for name, result in zip(string_paths, results):
    if isinstance(result, Exception):
            print("Failed to upload {} due to exception: {}".format(name, result))
    else:
        print("Uploaded {} to {}.".format(name, bucket.name))

Uploaded translated_piqa_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_lambada_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_wikiqa_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_dolly_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_mlqa_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_qa_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_joke_explaination_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_cot_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_mintaka_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_cnn_dailymail_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_soda_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_nqopen_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_wiki_split_info.jsonl to c4ai-instruct-multilingual-data.
Uploaded tran

## Rename data Files

In [13]:
aya_collection_data_dir = aya_collection_data_dir_translated #aya_collection_data_dir_templated

data_directory_path = Path(aya_collection_data_dir)
print(data_directory_path)

../aya-data-notebooks/AYA-collection-prep/translated-v2/data


In [14]:
# data_file_paths = [x for x in data_directory_path.glob("*.jsonl") if not x.name.startswith(".ipynb_checkpoints")] # for templated
data_file_paths = [x for x in data_directory_path.glob("*.jsonl") if not x.name.startswith(".ipynb_checkpoints")] #and x.name.startswith("mm_")]
data_file_paths

[PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_xlel_wd_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_piqa_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_flan_qa_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_flan_gem_wiki_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_paws_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_flan_coqa_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_mlqa_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_wiki_split_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-collection-prep/translated-v2/data/translated_flan_lambada_data.jsonl'),
 PosixPath('../aya-data-notebooks/AYA-c

In [44]:
# Rename file paths to remove additional "u_" if present
# data_file_paths = [path.rename(Path(path.parent, path.stem.replace("u_","") + path.suffix)) for path in data_file_paths]
# data_file_paths

[PosixPath('collection-v0/release-v0-vm/data/TeluguRiddles_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/LLM-japanese-vanilla-inst_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/NusaX-senti_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/thirukkural_QA_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/Mintaka_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/masakhane-news_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/AfriSenti_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/Wiki_split_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/indo_instruct_stories_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/xlel_wd_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/telugu-poems_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/joke_explaination_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/data/telugu-news-articles_data.jsonl'),
 PosixPath('collection-v0/release-v0-vm/

In [15]:
relative_paths = [path.relative_to(data_directory_path) for path in data_file_paths]
relative_paths

[PosixPath('translated_xlel_wd_data.jsonl'),
 PosixPath('translated_piqa_data.jsonl'),
 PosixPath('translated_flan_qa_data.jsonl'),
 PosixPath('translated_flan_gem_wiki_data.jsonl'),
 PosixPath('translated_paws_data.jsonl'),
 PosixPath('translated_flan_coqa_data.jsonl'),
 PosixPath('translated_mlqa_data.jsonl'),
 PosixPath('translated_wiki_split_data.jsonl'),
 PosixPath('translated_flan_lambada_data.jsonl'),
 PosixPath('translated_dolly_data.jsonl'),
 PosixPath('translated_adversarial_qa_data.jsonl'),
 PosixPath('translated_mintaka_data.jsonl'),
 PosixPath('translated_hotpotqa_data.jsonl'),
 PosixPath('translated_nqopen_data.jsonl'),
 PosixPath('translated_wikiqa_data.jsonl'),
 PosixPath('translated_soda_data.jsonl'),
 PosixPath('translated_cnn_dailymail_data.jsonl'),
 PosixPath('translated_flan_cot_data.jsonl'),
 PosixPath('translated_joke_explaination_data.jsonl')]

In [16]:
string_paths = [str(path) for path in relative_paths]

print("Found {} files.".format(len(string_paths)))

Found 19 files.


In [17]:
# push all data files

#"aya-collection-staging/v0/data/" --> v0 path

# v1 path for templated -> "aya-collection-staging/v1/templated_datasets/data/"

# v1 path for translated Main data (mm_) --> 

# v1 path for translated data -> "aya-collection-staging/v1/translated_datasets/data/"

results = transfer_manager.upload_many_from_filenames(
    bucket, string_paths, source_directory=aya_collection_data_dir, max_workers = 8, blob_name_prefix="aya-collection-staging/v2/translated_datasets/data/")

for name, result in zip(string_paths, results):
    if isinstance(result, Exception):
            print("Failed to upload {} due to exception: {}".format(name, result))
    else:
        print("Uploaded {} to {}.".format(name, bucket.name))

Uploaded translated_xlel_wd_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_piqa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_qa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_gem_wiki_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_paws_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_coqa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_mlqa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_wiki_split_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_flan_lambada_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_dolly_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_adversarial_qa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_mintaka_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded translated_hotpotqa_data.jsonl to c4ai-instruct-multilingual-data.
Uploaded tra